In [1]:
import json
from pymongo import MongoClient
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

In [ ]:
# === Configuración ===
json_file_path = "arxiv-metadata-oai-snapshot.json"
batch_size = 2000   # ¡Aumentar considerablemente! Puedes probar con 1000, 2000, 3000 o incluso más.
max_workers = 8     # Aumentar, idealmente cerca del número de núcleos lógicos de tu CPU. Prueba con 6, 8 o 12.
# Mantén max_pending_batches como un múltiplo de max_workers.
max_pending_batches = max_workers * 2 # Si max_workers es 8, esto sería 16

In [3]:
# === Conectar a MongoDB ===
client = MongoClient("mongodb://mongo1:30001,mongo2:30002,mongo3:30003/?replicaSet=my-replica-set&readPreference=primary&appname=MongoDB%20Compass&ssl=false")
db = client["arxiv_db"]
collection = db["articles"]

In [4]:
# === Función para insertar lote ===
def insert_batch(batch):
    try:
        collection.insert_many(batch, ordered=False)
    except Exception as e:
        print("Error al insertar batch:", e)

In [5]:
# === Contar líneas para tqdm ===
def count_lines(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        return sum(1 for _ in f)

total_lines = count_lines(json_file_path)

In [ ]:
# === Lectura + carga paralela ===
with open(json_file_path, 'r', encoding='utf-8') as f, tqdm(desc="Cargando artículos") as pbar:
    current_batch = []
    pending_futures = [] # Lista para almacenar los objetos Future

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        for line_num, line in enumerate(f): # line_num puede ser útil para depuración
            try:
                record = json.loads(line)
                # 👉 Agregar campo pdf_source
                record["pdf_source"] = f"https://arxiv.org/pdf/{record['id']}"
                current_batch.append(record)

                if len(current_batch) >= batch_size:
                    # Envia el lote para inserción y guarda el Future
                    future = executor.submit(insert_batch, list(current_batch)) # Se pasa una copia del batch
                    pending_futures.append(future)
                    current_batch = [] # Reinicia el lote

                    # === Control de flujo: Esperar si hay demasiados lotes pendientes ===
                    if len(pending_futures) >= max_pending_batches:
                        # Espera a que las tareas más antiguas se completen
                        # as_completed devuelve los futures a medida que se completan
                        for completed_future in as_completed(pending_futures):
                            try:
                                completed_future.result() # Obtiene el resultado (o re-lanza excepción)
                            except Exception as e:
                                print(f"Error en una tarea completada: {e}")
                            # Una vez que una tarea se completa, la eliminamos de la lista
                            pending_futures.remove(completed_future)
                            # Si ya no tenemos demasiados lotes pendientes, podemos seguir enviando
                            if len(pending_futures) < max_pending_batches:
                                break # Salir del bucle interno y continuar leyendo líneas

            except json.JSONDecodeError:
                # Omitir líneas que no son JSON válido, pero considera registrarlo si es inesperado
                # print(f"Saltando línea JSON mal formada en la línea {line_num + 1}: {line.strip()}")
                continue
            pbar.update(1) # Actualiza la barra de progreso por cada línea procesada

        # === Último batch (si queda algo después del bucle) ===
        if current_batch:
            future = executor.submit(insert_batch, list(current_batch))
            pending_futures.append(future)

        # === Esperar a que terminen todas las cargas restantes ===
        # Fuera del bucle principal, esperamos a que todas las tareas pendientes finalicen
        for completed_future in as_completed(pending_futures):
            try:
                completed_future.result()
            except Exception as e:
                print(f"Error en una tarea final pendiente: {e}")

print("✅ Carga paralela completa.")

Cargando artículos: 4500it [00:30, 144.79it/s]

Error al insertar batch: mongo2:30002: [Errno -3] Temporary failure in name resolution (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms),mongo1:30001: [Errno -3] Temporary failure in name resolution (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms),mongo3:30003: [Errno -3] Temporary failure in name resolution (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 6845f456f8a3717a58fa9f0f, topology_type: ReplicaSetNoPrimary, servers: [<ServerDescription ('mongo1', 30001) server_type: Unknown, rtt: None, error=AutoReconnect('mongo1:30001: [Errno -3] Temporary failure in name resolution (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>, <ServerDescription ('mongo2', 30002) server_type: Unknown, rtt: None, error=AutoReconnect('mongo2:30002: [Errno -3] Temporary failure in name resolution (configured timeouts: socketT

Cargando artículos: 4800it [00:30, 155.33it/s]

Error al insertar batch: mongo2:30002: [Errno -3] Temporary failure in name resolution (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms),mongo1:30001: [Errno -3] Temporary failure in name resolution (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms),mongo3:30003: [Errno -3] Temporary failure in name resolution (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 6845f456f8a3717a58fa9f0f, topology_type: ReplicaSetNoPrimary, servers: [<ServerDescription ('mongo1', 30001) server_type: Unknown, rtt: None, error=AutoReconnect('mongo1:30001: [Errno -3] Temporary failure in name resolution (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>, <ServerDescription ('mongo2', 30002) server_type: Unknown, rtt: None, error=AutoReconnect('mongo2:30002: [Errno -3] Temporary failure in name resolution (configured timeouts: socketT

Cargando artículos: 5400it [01:00, 65.83it/s] 

...

Error al insertar batch: mongo2:30002: [Errno -3] Temporary failure in name resolution (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms),mongo1:30001: [Errno -3] Temporary failure in name resolution (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms),mongo3:30003: [Errno -3] Temporary failure in name resolution (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 6845f456f8a3717a58fa9f0f, topology_type: ReplicaSetNoPrimary, servers: [<ServerDescription ('mongo1', 30001) server_type: Unknown, rtt: None, error=AutoReconnect('mongo1:30001: [Errno -3] Temporary failure in name resolution (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>, <ServerDescription ('mongo2', 30002) server_type: Unknown, rtt: None, error=AutoReconnect('mongo2:30002: [Errno -3] Temporary failure in name resolution (configured timeouts: socketT

Cargando artículos: 66900it [35:07, 33.92it/s]

Error al insertar batch: mongo2:30002: [Errno -3] Temporary failure in name resolution (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms),mongo1:30001: [Errno -3] Temporary failure in name resolution (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms),mongo3:30003: [Errno -3] Temporary failure in name resolution (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 6845f456f8a3717a58fa9f0f, topology_type: ReplicaSetNoPrimary, servers: [<ServerDescription ('mongo1', 30001) server_type: Unknown, rtt: None, error=AutoReconnect('mongo1:30001: [Errno -3] Temporary failure in name resolution (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>, <ServerDescription ('mongo2', 30002) server_type: Unknown, rtt: None, error=AutoReconnect('mongo2:30002: [Errno -3] Temporary failure in name resolution (configured timeouts: socketT

Cargando artículos: 67499it [35:20, 33.92it/s]

Error al insertar batch: mongo2:30002: [Errno -3] Temporary failure in name resolution (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms),mongo1:30001: [Errno -3] Temporary failure in name resolution (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms),mongo3:30003: [Errno -3] Temporary failure in name resolution (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 6845f456f8a3717a58fa9f0f, topology_type: ReplicaSetNoPrimary, servers: [<ServerDescription ('mongo1', 30001) server_type: Unknown, rtt: None, error=AutoReconnect('mongo1:30001: [Errno -3] Temporary failure in name resolution (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>, <ServerDescription ('mongo2', 30002) server_type: Unknown, rtt: None, error=AutoReconnect('mongo2:30002: [Errno -3] Temporary failure in name resolution (configured timeouts: socketT

In [13]:
'''
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27018/")
db = client["arxiv_db"]
collection = db["articles"]

# ⚠️ Eliminar todos los documentos
collection.delete_many({})
print("🗑️ Todos los documentos eliminados.")
'''

🗑️ Todos los documentos eliminados.
